In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import math as mt
import datetime as dt
from sklearn.cluster import KMeans
import random as ran
from sklearn.metrics.pairwise import euclidean_distances as euc
from functools import reduce

# 한글출력
matplotlib.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

from src import crs
from IPython.display import clear_output, Markdown, display

In [4]:
# 1. load_excel
data_path = "data/apt_all.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

In [5]:
# 2. data preprocessing
p, m = crs.utils.data_preprocessing(xlsx)

m.set_index("month", inplace=True)
m.head()

/Users/formegusto/Desktop/idoformegusto/development/research/what-is-public-fee/src/crs/utils/data_preprocessing.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  datas_df[household_name] = household_df[col][3:].to_list()
/Users/formegusto/Desktop/idoformegusto/development/research/what-is-public-fee/src/crs/utils/data_preprocessing.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  month_usage_df[name] = [


,아파트1-101-1002,아파트1-101-104,아파트1-101-1102,아파트1-101-1202,아파트1-101-1302,아파트1-101-1401,아파트1-101-1403,아파트1-101-1502,아파트1-101-1504,아파트1-101-1602,...,아파트4-104-101,아파트4-104-1301,아파트4-104-1501,아파트4-104-301,아파트4-105-102,아파트4-105-103,아파트4-105-1102,아파트4-105-1104,아파트4-105-601,아파트4-105-803
month,,,,,,,,,,,,,,,,,,,,,
1,253,689,183,239,455,343,380,219,389,350,...,335,363,181,216,270,518,230,279,161,156
2,251,481,177,218,397,297,339,197,348,288,...,309,339,146,184,267,344,208,236,167,150
3,255,424,203,222,378,303,349,202,357,368,...,313,372,152,236,268,417,212,272,166,155
4,252,355,180,230,368,278,298,188,346,365,...,266,346,149,210,268,421,208,243,156,148
5,230,357,258,203,414,270,418,156,320,265,...,305,256,151,222,287,349,185,284,165,180


# 임의 가구 구성 셋팅

In [34]:
_m = m.sample(n=50, axis=1)

_m.head()

# 초기 정보 셋팅
- 월, 공용부 차지 percentage

In [275]:
_month = 1

month_df = pd.DataFrame(_m.loc[_month])
month_df.reset_index(inplace=True)

month_df.columns = ['name', 'usage (kWh)']

households_kWh = month_df['usage (kWh)'].sum()
households_count = len(month_df)
PUBLIC_PERCENTAGE = 30
APT = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))

APT_mean = round(APT / households_count)
households_mean = round(month_df['usage (kWh)'].mean())

display(Markdown(
    "**설정 월은 {}월 입니다.**<br/>".format(_month) +
    "• 가구 수 : {}가구<br/>".format(households_count) +
    "• APT 총 사용량 : {}kWh (평균사용량:{}kWh)<br/>".format(format(APT,","),
                                                   APT_mean) +
    "• 세대부 총 사용량 : {}kWh (평균사용량:{}kWh)".format(format(households_kWh, ","),
                                              households_mean)
))

month_df.head()

**설정 월은 1월 입니다.**<br/>• 가구 수 : 50가구<br/>• APT 총 사용량 : 19,613kWh (평균사용량:392kWh)<br/>• 세대부 총 사용량 : 13,729kWh (평균사용량:275kWh)

,name,usage (kWh)
0,아파트1-104-408,39
1,아파트4-102-104,119
2,아파트2-3-603,138
3,아파트4-103-103,331
4,아파트1-103-903,273


# 청구서 셋팅

In [276]:
calc = crs.models.ManagementOffice(
        month=_month,
        households=month_df,
        APT=APT,
        contract="단일계약"
    )
fee = calc.fee
fee

[[0, 200, 730, 73.3], [201, 400, 1260, 142.3], [401, inf, 6060, 210.6]]

# 기본요금 예측

## 누진단계 셋팅

In [277]:
nugin_steps = [_[0] for _ in fee]
nugin_basic = [_[2] for _ in fee]

## 누진단계 분할

In [317]:
nugin_dist = pd.DataFrame(columns=['가구 수'])

for step, now_kWh in enumerate(nugin_steps):    
    if step < 2:
        next_kWh = nugin_steps[step + 1]
        count = len(month_df[(month_df['usage (kWh)'] >= now_kWh) &\
                             (month_df['usage (kWh)'] < next_kWh)])
        nugin_dist.loc['{} 단계'.format(step + 1)] = [count]
                    
    else:
        count = len(month_df[month_df['usage (kWh)'] >= now_kWh])
        nugin_dist.loc['{} 단계'.format(step + 1)] = [count]
        
nugin_dist.head()

,가구 수
1 단계,13
2 단계,32
3 단계,5


## 수거액 산정

In [318]:
collect_basic = list()

for idx,count in enumerate(nugin_dist['가구 수'].values):
    basic = nugin_basic[idx]
    
    collect_basic.append(basic * count)
    
nugin_dist['수거액'] = collect_basic

nugin_dist.head()

,가구 수,수거액
1 단계,13,9490
2 단계,32,40320
3 단계,5,30300


## APT 누진 단계 확인 및 기본요금 확인

In [319]:
print("현재 APT 평균사용량은 {}kWh 입니다.".format(APT_mean))
print("요금표 {}에 근거하여,".format(fee))

for step, _f in enumerate(fee):
    if APT_mean < _f[1]:
        apt_step = step
        apt_basic = _f[2] * households_count
        break
        
print("\n현재 APT 평균사용량의 누진단계는 {}단계 입니다.".format(apt_step + 1))
print("{} 기본요금이 적용되어, 기본요금은 {} * {}가구 수 = {}원 입니다.".format(
    format(fee[apt_step][2], ","),
    format(fee[apt_step][2], ","),
    households_count,
    format(apt_basic, ",")
))

현재 APT 평균사용량은 392kWh 입니다.
요금표 [[0, 200, 730, 73.3], [201, 400, 1260, 142.3], [401, inf, 6060, 210.6]]에 근거하여,

현재 APT 평균사용량의 누진단계는 2단계 입니다.
1,260 기본요금이 적용되어, 기본요금은 1,260 * 50가구 수 = 63,000원 입니다.


## 공용부 기본요금 산출
- APT 기본요금 - 세대부 수거액

In [320]:
households_basic = nugin_dist['수거액'].sum()
public_basic = apt_basic - households_basic

print("APT 기본요금 : {}원".format(format(apt_basic, ",")))
print("세대부 기본요금 : {}원".format(format(households_basic, ",")))
print("공용부 기본요금 : {}원".format(format(public_basic, ",")))

APT 기본요금 : 63,000원
세대부 기본요금 : 80,110원
공용부 기본요금 : -17,110원


# 전력량 요금 예측

## 누진단계별 수거해야 하는 사용량

In [321]:
col_list = list()
_usage = households_mean
for step, _f in enumerate(fee):
    collect_rate = _f[1] - _f[0]

    ## 2단계일 경우
    if step == 1:
        collect_rate += 1

    if households_mean < _f[1]:
        if (step == 0) or (step == 1):
            _col = _usage % collect_rate

        else:
            _col = _usage
    else:
        _col = collect_rate

    if _usage == 0:
        break
    else:
        col_list.append(_col * households_count)
        print(_col)
        _usage -= _col

apt_col = pd.DataFrame(col_list, columns=['수거 필요 액수'])
apt_col.index = pd.Series(range(1, len(col_list) + 1), name="누진 단계")

apt_col

200
75


,수거 필요 액수
누진 단계,
1,10000
2,3750


## 가구별 누진 단계 수거 사용량

In [322]:
month_df.head(6)

,name,usage (kWh)
0,아파트1-104-408,39
1,아파트4-102-104,119
2,아파트2-3-603,138
3,아파트4-103-103,331
4,아파트1-103-903,273
5,아파트1-104-203,197


In [323]:
# nugin elec rate collect
nerc = pd.DataFrame(columns=['가구명', '누진단계', '수거액'])

for idx, row in month_df.head(100).iterrows():
    name, usage = row
    
    col_list = list()
    _usage = usage
    for step, _f in enumerate(fee):
        collect_rate = _f[1] - _f[0]
            
        ## 2단계일 경우
        if step == 1:
            collect_rate += 1

        if usage < _f[1]:
            if (step == 0) or (step == 1):
                _col = _usage % collect_rate
                
            else:
                _col = _usage
        else:
            _col = collect_rate
            
        if _usage == 0:
            break
        else:
            col_list.append(_col)
            _usage -= _col
            
    for step,col in enumerate(col_list):
        nerc = nerc.append({
            "가구명": name,
            "누진단계": step + 1,
            "수거액": col
        }, ignore_index=True)
        
nerc = nerc.pivot_table(index=['누진단계'], values=['수거액'], aggfunc='sum')
nerc

,수거액
누진단계,
1,9155
2,4203
3,371


### 가구 수거액 확인

In [324]:
col_pd = pd.concat([apt_col, nerc], axis=1).fillna(0).astype("int")
col_pd['오차'] = col_pd['수거 필요 액수'] - col_pd['수거액']

col_pd

,수거 필요 액수,수거액,오차
1,10000,9155,845
2,3750,4203,-453
3,0,371,-371


## 공용부 요금 시작 지점 확인

In [325]:
public_start = 0

for step, row in col_pd.iterrows():
    _f = fee[step - 1]
    rate = row['오차']
    
    public_start += (_f[3] * rate)
    
public_start = round(public_start)

print("공용부 시작 전력량 요금은 {}원 입니다.".format(format(public_start, ",")))

공용부 시작 전력량 요금은 -80,656원 입니다.


## 공용부 시작 누진 단계 확인

In [326]:
# 복구 로직
APT_mean = round(APT / households_count)
households_mean = round(month_df['usage (kWh)'].mean())

In [327]:
col_list = list()
_usage = APT_mean
for step, _f in enumerate(fee):
    collect_rate = _f[1] - _f[0]

    ## 2단계일 경우
    if step == 1:
        collect_rate += 1

    if APT_mean < _f[1]:
        if (step == 0) or (step == 1):
            _col = _usage % collect_rate
        else:
            _col = _usage
    else:
        _col = collect_rate

    if _usage == 0:
        break
    else:
        col_list.append(_col)
        _usage -= _col
        
_usage = households_mean
for idx,col in enumerate(col_list):
    if _usage < col:
        col_list[idx] -= _usage
        _usage -= _usage
    else:
        _usage -= col_list[idx]
        col_list[idx] = 0
        
    if _usage == 0:
        break
        
elec_rate_start = np.where(np.array(col_list) != 0)[0][0]
print("공용부 전력량 요금은 {}단계부터 시작합니다. 해당 단계의 나머지 사용량은 {}kWh입니다.".format(
    elec_rate_start + 1,
    col_list[elec_rate_start]
))

공용부 전력량 요금은 2단계부터 시작합니다. 해당 단계의 나머지 사용량은 117kWh입니다.


In [328]:
public_elec_rate = 0

for step,col in enumerate(col_list):
    _f = fee[step]
    public_elec_rate += _f[3] * col * households_count

public_elec_rate = round(public_elec_rate) + public_start
print("공용부 전력량 요금: {}원".format(format(public_elec_rate, ",")))

공용부 전력량 요금: 751,799원


In [329]:
public_env = round((APT - households_kWh) * 5.3)
public_fuel = round((APT - households_kWh) * 3)

print("공용부 기후 환경 요금 : {}원".format(format(public_env, ",")))
print("공용부 연료비 조정액 : {}원".format(format(public_fuel, ",")))

공용부 기후 환경 요금 : 31,185원
공용부 연료비 조정액 : 17,652원


In [330]:
public_elec_bill = public_basic + public_elec_rate + public_env - public_fuel
households_guarantees = sum([_.guarantee for _ in calc.households])

public_elec_bill += households_guarantees

print("공용부 전기요금계 : {}원".format(format(public_elec_bill, ",")))

공용부 전기요금계 : 780,722원


In [331]:
public_vat = round(public_elec_bill * 0.1)
public_fund = round(public_elec_bill * 0.037)

public_bill = public_elec_bill + public_vat + public_fund
print("공용부 예측 최종청구금액 : {}원".format(format(public_bill, ",")))

공용부 예측 최종청구금액 : 887,681원


In [332]:
calc.public_bill

885830